In [1]:
from hypothesis.extra.pandas import indexes, series
import hypothesis.strategies as st
from hypothesis import given, assume

In [2]:
series(dtype=float).example()

0    1.199194e+16
1    1.199194e+16
2    1.199194e+16
dtype: float64

In [3]:
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu, norm


def AUC(labels: pd.Series, values: pd.Series) -> Dict[str, float]:
    labels = labels.fillna(value=0)
    values = values.fillna(value=0)
    posii = labels[labels > 0]
    negii = labels[labels <= 0]
    posn = len(posii)
    negn = len(negii)
    posval = values[posii.index]
    negval = values[negii.index]
    if posn > 0 and negn > 0:
        statistic, pvalue = mannwhitneyu(posval, negval, alternative="greater")
        conf_int_low, conf_int_high = mannwhitneyu_conf_int(posval, negval)
        res = {
            "low": conf_int_low,
            "high": conf_int_high,
            "auc": (statistic / (posn * negn)),
            "pval": pvalue,
        }
    else:
        res = {"auc": 0.5, "pval": np.nan}

    return res


def mannwhitneyu_conf_int(
    x: np.array, y: np.array, alpha: float = 0.05
) -> Tuple[float, float]:
    n = len(x)
    m = len(y)

    N = norm.ppf(1 - alpha / 2)

    diffs = sorted([i - j for i in x for j in y])

    # For an approximate 100(1-a)% confidence interval first calculate K:
    nm = n * m
    top = nm * (n + m + 1)
    right = N * np.sqrt(top / 12)
    left = (n * m) / 2
    K = left - right

    # The Kth smallest to the Kth largest of the n x m differences
    # lx and ly should be > ~20
    return (diffs[round(K)], diffs[len(diffs) - round(K)])

In [4]:
import hypothesis.strategies as st
from hypothesis import given, assume

In [5]:
@given(
    x=st.lists(elements=st.integers(), min_size=20),
    y=st.lists(elements=st.integers(), min_size=20),
)
def testconfint(x, y):
    assume(len(x) > 5)
    assume(len(x) > 5)
    mannwhitneyu_conf_int(x, y)

In [6]:
testconfint()

In [7]:
from hypothesis.extra.pandas import series, indexes

In [8]:
@given(x=series(dtype=float), y=series(dtype=float))
def testAUC(x, y):
    assume(len(x) == len(y))
    assume(len(x) > 5)
    assume(len(y) > 5)
    assume(x.index.isin(y.index).all())
    assume(y.index.isin(x.index).all())
    res = AUC(x, y)
    assert type(res) == "dict"

In [9]:
testAUC()

Falsifying example: testAUC(
    x=0    0.0
    1    1.0
    2    1.0
    3    1.0
    4    1.0
    5    1.0
    dtype: float64, y=0    0.0
    1    0.0
    2    0.0
    3    0.0
    4    0.0
    5    0.0
    dtype: float64,
)
Traceback (most recent call last):
  File "/tmp/ipykernel_8200/377233995.py", line 8, in testAUC
    res = AUC(x, y)
  File "/tmp/ipykernel_8200/2590556554.py", line 18, in AUC
    conf_int_low, conf_int_high = mannwhitneyu_conf_int(posval, negval)
  File "/tmp/ipykernel_8200/2590556554.py", line 55, in mannwhitneyu_conf_int
    return (diffs[round(K)], diffs[len(diffs)-round(K)])
IndexError: list index out of range

Falsifying example: testAUC(
    x=0    0.0
    1    0.0
    2    0.0
    3    0.0
    4    0.0
    5    0.0
    dtype: float64, y=0    0.0
    1    0.0
    2    0.0
    3    0.0
    4    0.0
    5    0.0
    dtype: float64,
)
Traceback (most recent call last):
  File "/tmp/ipykernel_8200/377233995.py", line 9, in testAUC
    assert(type(res) == "dic

MultipleFailures: Hypothesis found 2 distinct failures.

In [10]:
import importlib.resources

In [14]:
from pathlib import Path

In [15]:
Path.cwd()

PosixPath('/home/milo/workspace/pyplier/tests/data/AUC')

In [16]:
labels_file = Path("labels.csv")
values_file = Path("values.csv")

In [24]:
with values_file as vf, labels_file as lf:
    values_df = pd.read_csv(vf, index_col=0)
    labels_df = pd.read_csv(lf, index_col=0)

In [25]:
labels_df

,REACTOME_GENERIC_TRANSCRIPTION_PATHWAY
index,
GAS6,0
MMP14,0
MARCKSL1,0
SPARC,0
CTSD,0
...,...
TRPM4,0
LAIR2,0
ZNF135,0


In [29]:
test_res = AUC(labels_df.iloc[:, 0], values_df.iloc[:, 0])

In [31]:
from joblib import dump

In [32]:
dump(test_res, "auc_test_result.pkl")

['auc_test_result.pkl']

In [27]:
labels = labels_df.fillna(value=0)
values = values_df.fillna(value=0)

NameError: name 'labels' is not defined

In [ ]:
posii = labels[labels > 0]
negii = labels[labels <= 0]

In [ ]:
labels = labels.fillna(value=0)
values = values.fillna(value=0)
posii = labels[labels > 0]
negii = labels[labels <= 0]
posn = len(posii)
negn = len(negii)
posval = values[posii.index]
negval = values[negii.index]
if posn > 0 and negn > 0:
    statistic, pvalue = mannwhitneyu(posval, negval, alternative="greater")
    conf_int_low, conf_int_high = mannwhitneyu_conf_int(posval, negval)
    res = {
        "low": conf_int_low,
        "high": conf_int_high,
        "auc": (statistic / (posn * negn)),
        "pval": pvalue,
    }
else:
    res = {"auc": 0.5, "pval": np.nan}